## Stage 2: Tool Use / Function Calling

It's time to give agent the ability to call real-world functions like fetching data or executing code when needed, not just describe what it would do.

### Concept Behind Tool Use:

LLMs are good at reasoning, but they can't directly:

- access APIs

- run Python

- browse the web, unless we build that in.


In [4]:
# step 1 - Define a Tool
def add(a, b):
    return a + b

In [5]:
# describe this tool to LLM in JSON schema
tools = [
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "Add two numbers and return the result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "The first number"},
                    "b": {"type": "number", "description": "The second number"},
                },
            },
            "required": ["a", "b"],
        },
    }
]

In [6]:
%pip install together python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from together import Together
from dotenv import load_dotenv

In [10]:
client = Together()
load_dotenv()
response = client.chat.completions.create(
    model="meta-llama/Llama-3-70B-Instruct",
    messages=[{"role": "user", "content": "What is 5 plus 7?"}],
    tools=tools,
    tool_choice="auto",
)

msg = response.choices[0].message
print("LLM:", msg.content)
print("Tool call:", msg.tool_calls)

AuthenticationError: The api_key client option must be set either by passing api_key to the client or by setting the TOGETHER_API_KEY environment variable